In [1]:
import pickle
import pandas

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn import metrics



In [19]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_preprocessed.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_preprocessed.pickle", "rb"))

In [20]:
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = TfidfVectorizer() # or term frequency

X_train_tf = tf_vectorizer.fit_transform(x_train)

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 13.809509 seconds
n_samples: 560000, n_features: 698699


In [21]:
t = time()
X_test_tf = tf_vectorizer.transform(x_test)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 1.600998 seconds
n_samples: 70000, n_features: 698699


In [5]:
import pandas
import numpy

In [25]:
y_train = [y-1 for y in y_train]
y_train[:5]

[0, 0, 0, 0, 0]

In [7]:
import xgboost as xgb

from sklearn.metrics import classification_report, confusion_matrix

In [28]:
y_test = [y-1 for y in y_test]

In [9]:
dtrain = xgb.DMatrix(data=X_train_tf, label=y_train)
dtest = xgb.DMatrix(data=X_test_tf)

In [10]:
import gc
import sys
del x_train
del x_test
del X_train_tf
del X_test_tf
del tf_vectorizer
gc.collect()

0

In [13]:
from xgboost import XGBClassifier
from time import time


params = {
    'max_depth': 6,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 14,
    # Set number of GPUs if available   
#     'tree_method': 'gpu_hist',
#     'gpu_id': 0
}
t = time()
bst = xgb.train(params, dtrain)

training_time = time() - t
print("train time: %0.3fs" % training_time)


t = time()
pred = bst.predict(dtest)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

print(classification_report(y_test, pred))
cm = confusion_matrix(y_test, pred)
cm

train time: 190.961s
test time:  0.094s
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      5000
           1       0.93      0.96      0.95      5000
           2       0.72      0.80      0.76      5000
           3       0.96      0.94      0.95      5000
           4       0.95      0.90      0.92      5000
           5       0.93      0.91      0.92      5000
           6       0.93      0.88      0.90      5000
           7       0.96      0.95      0.95      5000
           8       0.99      0.99      0.99      5000
           9       0.89      0.93      0.91      5000
          10       0.96      0.89      0.93      5000
          11       0.95      0.98      0.96      5000
          12       0.95      0.97      0.96      5000
          13       0.91      0.92      0.91      5000

    accuracy                           0.92     70000
   macro avg       0.92      0.92      0.92     70000
weighted avg       0.92      0.92      0

array([[4247,   65,  167,   45,   37,  112,  118,   19,    6,   22,    9,
          33,   38,   82],
       [  42, 4825,   22,   13,   11,   15,   46,    7,    3,    0,    1,
           0,    9,    6],
       [  92,   98, 4016,   78,   91,   18,   20,    9,    4,   14,    5,
         181,   97,  277],
       [   8,   13,  212, 4701,   15,   23,    2,    9,    0,    9,    2,
           0,    2,    4],
       [  47,   91,  273,   23, 4488,   15,   10,   14,    3,   10,    5,
           1,    3,   17],
       [ 178,    6,  135,    7,   31, 4562,   22,   28,    0,   14,    7,
           3,    4,    3],
       [ 107,   78,  113,    3,   41,  115, 4392,   79,   28,   18,    3,
           2,    9,   12],
       [   8,    6,   85,    1,    7,   16,   91, 4747,   21,   13,    3,
           0,    0,    2],
       [   0,    4,   11,    0,    0,    0,   17,   10, 4955,    2,    0,
           0,    1,    0],
       [   8,    0,  200,    0,    1,    7,    1,   14,    0, 4640,  124,
           0,    

In [14]:
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import precision_score

In [15]:
score1 = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score1)

accuracy:   0.919


In [16]:
import lightgbm as lgb

In [30]:
d_train=lgb.Dataset(X_train_tf, label=y_train)
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_leaves'] = 1100
params['num_class']=14 #no.of unique values in the target class not inclusive of the end value
params['device'] = 'gpu'
t = time()
clf=lgb.train(params,d_train,100)
training_time = time() - t
print("train time: %0.3fs" % training_time)


t = time()
y_pred_1=clf.predict(X_test_tf)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 941844
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 39429
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1050 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start t

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [31]:

y_pred_1 = [numpy.argmax(line) for line in y_pred_1]#printing the predictions

print(precision_score(y_pred_1,y_test,average=None).mean())

print(classification_report(y_test, y_pred_1))
cm = confusion_matrix(y_test, y_pred_1)
cm

0.951
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      5000
           1       0.96      0.97      0.97      5000
           2       0.85      0.87      0.86      5000
           3       0.97      0.97      0.97      5000
           4       0.96      0.94      0.95      5000
           5       0.95      0.95      0.95      5000
           6       0.95      0.93      0.94      5000
           7       0.97      0.97      0.97      5000
           8       0.99      0.99      0.99      5000
           9       0.95      0.97      0.96      5000
          10       0.98      0.95      0.96      5000
          11       0.97      0.98      0.97      5000
          12       0.97      0.98      0.97      5000
          13       0.93      0.95      0.94      5000

    accuracy                           0.95     70000
   macro avg       0.95      0.95      0.95     70000
weighted avg       0.95      0.95      0.95     70000



array([[4488,   60,   87,   33,   27,   92,   72,   10,    5,    9,    5,
          25,   31,   56],
       [  33, 4874,   19,    5,    8,    6,   39,    1,    3,    1,    1,
           0,    7,    3],
       [  58,   42, 4358,   48,   80,   12,   11,    6,    0,    7,    1,
         113,   61,  203],
       [   6,    5,  117, 4828,   14,   11,    4,    4,    0,    4,    1,
           0,    3,    3],
       [  37,   40,  163,   24, 4686,   14,    8,    4,    3,    4,    1,
           1,    2,   13],
       [ 113,    0,   50,    4,   19, 4764,   21,   13,    0,    3,    1,
           0,    4,    8],
       [  84,   57,   36,    0,   28,   61, 4634,   59,   17,    6,    2,
           2,    7,    7],
       [  13,    2,   29,    0,    7,    4,   43, 4870,   18,    9,    3,
           0,    0,    2],
       [   3,    3,    3,    0,    0,    0,   15,   10, 4963,    1,    0,
           1,    0,    1],
       [   5,    0,   30,    0,    1,    8,    1,   14,    0, 4851,   88,
           0,    

In [32]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

In [33]:

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler


In [34]:
del d_train
gc.collect()

36

In [47]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_preprocessed.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))


y_train = y_train[slice(0, len(y_train), 5)]
x_train = x_train[slice(0, len(x_train), 5)]

In [56]:
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_preprocessed.pickle", "rb"))

In [48]:
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = TfidfVectorizer() # or term frequency

x_train = tf_vectorizer.fit_transform(x_train)

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % x_train.shape)

t = time()
x_test = tf_vectorizer.transform(x_test)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % x_test.shape)

Time taken to extract features from training data : 2.933693 seconds
n_samples: 112000, n_features: 243619
Time taken to extract features from test data : 11.374985 seconds
n_samples: 560000, n_features: 243619


In [57]:
x_test = tf_vectorizer.transform(x_test)


In [36]:
import tensorflow as tf


In [37]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



In [49]:



from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
# Choose either standardization or Normalization
# On this data Min Max Normalization is used because we need to fit Naive Bayes

# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=QuantileTransformer()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(x_train)


x_train = PredictorScalerFit.transform(x_train)


In [58]:
x_test = PredictorScalerFit.transform(x_test)

In [59]:
y_test_np = []
import numpy as np
for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test_np = np.asarray(y_test_np)
y_test_np.shape

(70000, 14)

In [51]:
y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train_np = np.asarray(y_train_np)
y_train_np.shape

(112000, 14)

In [52]:
x_train.shape

(112000, 243619)

In [66]:
import scipy.sparse as sps

In [71]:
x_train.flat[0].todense()

MemoryError: Unable to allocate 203. GiB for an array with shape (112000, 243619) and data type float64

In [67]:
x_train[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [64]:
x_train = np.asarray(x_train)

In [63]:
x_test

<70000x243619 sparse matrix of type '<class 'numpy.float64'>'
	with 1535949 stored elements in Compressed Sparse Column format>

In [55]:
callback = EarlyStopping(monitor='val_loss',patience=2, min_delta=0.0001)

checkpoint = ModelCheckpoint("best_model", monitor='loss', verbose=1, save_best_only=True, mode='min')

reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [61]:
model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(243619,), name='input'))
# model_l1.add(Conv1D(128, 5, activation='relu'))
# model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train_np, epochs=10, verbose=1, validation_split=0.2, batch_size=32
                   ,callbacks=[reduce,callback,checkpoint])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               31183360  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 14)                462       
                                                                 
Total params: 31,187,950
Trainable params: 31,187,950
Non-trainable params: 0
_________________________________________________________________


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'scipy.sparse.csc.csc_matrix'>]